# Trabalho 04

#### Marrielly Chrystina Martines



O objetivo desta tarefa é explorar o conjunto de dados "*agaricus_lepiota_small_c.csv*", que contém dados de informações sobre cogumelos, e implementar um sistema de classificação usando os classificadores KNN e CSV, que irão categorizar a amostra como sendo venenosa ou não. Será avaliado o desempenho de cada classificador por meio de validação cruzada em dois níveis e, posteriormente, será feito o Teste-T para mostrar se a diferença de desempenho dos classificadores foi significativa.

In [ ]:
import pandas as pd
import numpy as np

from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OrdinalEncoder, OneHotEncoder, StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.model_selection import train_test_split, StratifiedKFold, GridSearchCV
from sklearn.metrics import accuracy_score
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC

from tqdm.notebook import tqdm

from scipy.stats import ttest_ind_from_stats

In [ ]:
df = pd.read_csv('agaricus_lepiota_small_c.csv')
df

,class,cap-shape,cap-surface,cap-color,bruises,odor,gill-attachment,gill-spacing,gill-size,gill-color,...,stalk-surface-below-ring,stalk-color-above-ring,stalk-color-below-ring,veil-type,veil-color,ring-number,ring-type,spore-print-color,population,habitat
0,e,x,s,y,t,a,f,w,b,g,...,s,w,w,p,w,o,p,n,v,d
1,e,f,s,y,f,n,f,c,b,p,...,s,w,w,p,w,o,f,n,y,g
2,e,k,s,w,f,c,f,w,b,g,...,s,w,n,p,w,t,e,w,n,g
3,e,f,f,n,t,n,f,c,b,w,...,s,g,w,p,w,o,p,k,v,d
4,p,x,s,w,t,p,f,c,n,w,...,s,w,w,p,w,o,p,n,s,u
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,p,x,f,p,f,c,f,w,n,n,...,s,w,w,p,w,o,p,n,v,d
996,p,x,y,n,f,n,f,c,n,w,...,y,w,y,p,w,o,e,w,v,d
997,e,x,f,g,f,n,f,c,b,u,...,s,g,g,p,w,o,e,k,y,d
998,e,b,s,w,t,a,f,c,b,b,...,s,g,w,p,w,o,p,h,y,p


## Imputação dos dados

In [ ]:
nan = df.isnull()
nan.sum()

class                         0
cap-shape                     0
cap-surface                   0
cap-color                     0
bruises                       0
odor                          0
gill-attachment               0
gill-spacing                  0
gill-size                     0
gill-color                    0
stalk-shape                   0
stalk-root                  310
stalk-surface-above-ring      0
stalk-surface-below-ring      0
stalk-color-above-ring        0
stalk-color-below-ring        0
veil-type                     0
veil-color                    0
ring-number                   0
ring-type                     0
spore-print-color             0
population                    0
habitat                       0
dtype: int64

Nota-se que esta base de dados contém instâncias com valores desconhecidos para o atributo '*stalk-root*'. Como a quantidade de instâncias assim é significativamente alta, foi escolhido deletar a coluna referente.

In [ ]:
df.drop('stalk-root', axis=1, inplace=True)
df

,class,cap-shape,cap-surface,cap-color,bruises,odor,gill-attachment,gill-spacing,gill-size,gill-color,...,stalk-surface-below-ring,stalk-color-above-ring,stalk-color-below-ring,veil-type,veil-color,ring-number,ring-type,spore-print-color,population,habitat
0,e,x,s,y,t,a,f,w,b,g,...,s,w,w,p,w,o,p,n,v,d
1,e,f,s,y,f,n,f,c,b,p,...,s,w,w,p,w,o,f,n,y,g
2,e,k,s,w,f,c,f,w,b,g,...,s,w,n,p,w,t,e,w,n,g
3,e,f,f,n,t,n,f,c,b,w,...,s,g,w,p,w,o,p,k,v,d
4,p,x,s,w,t,p,f,c,n,w,...,s,w,w,p,w,o,p,n,s,u
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,p,x,f,p,f,c,f,w,n,n,...,s,w,w,p,w,o,p,n,v,d
996,p,x,y,n,f,n,f,c,n,w,...,y,w,y,p,w,o,e,w,v,d
997,e,x,f,g,f,n,f,c,b,u,...,s,g,g,p,w,o,e,k,y,d
998,e,b,s,w,t,a,f,c,b,b,...,s,g,w,p,w,o,p,h,y,p


In [ ]:
x = df.drop('class', axis=1)
y = df['class']

## Codificação dos Atributos Categóricos

Nesta etapa é realizado o mapeamento dos valores de atributos categóricos em valores numéricos. Foi utilizado o OrdinalEncoder, onde cada valor recebe um código numérico inteiro que o representa, para o atributo "ring-number", já que possui uma relação de ordem intrínseca. Para os demais atributos, por não possuírem tal característica, foi utilizado o OneHotEncoder, onde um atributo categórico com categorias é codificado em atributos binários, um deles com valor 1, que representa o valor do atributo para aquela instância, e os demais com valor 0.

In [ ]:
transformers = [
    ('oh_cap_shape', OneHotEncoder(), ['cap-shape']),
    ('oh_cap_surface', OneHotEncoder(), ['cap-surface']),
    ('oh_cap_color', OneHotEncoder(), ['cap-color']),
    ('oh_bruises', OneHotEncoder(), ['bruises']),
    ('oh_odor', OneHotEncoder(), ['odor']),
    ('oh_gill_attachment', OneHotEncoder(), ['gill-attachment']),
    ('oh_gill_spacing', OneHotEncoder(), ['gill-spacing']),
    ('oh_gill_size', OneHotEncoder(), ['gill-size']),
    ('oh_gill_color', OneHotEncoder(), ['gill-color']),
    ('oh_stalk_shape', OneHotEncoder(), ['stalk-shape']),
    ('oh_stalk_surface_above_ring', OneHotEncoder(), ['stalk-surface-above-ring']),
    ('oh_stalk_surface_below_ring', OneHotEncoder(), ['stalk-surface-below-ring']),
    ('oh_stalk_color_above_ring', OneHotEncoder(), ['stalk-color-above-ring']),
    ('oh_stalk_color_below_ring', OneHotEncoder(), ['stalk-color-below-ring']),
    ('oh_veil_type', OneHotEncoder(), ['veil-type']),
    ('oh_veil_color', OneHotEncoder(), ['veil-color']),
    ('oh_ring_type', OneHotEncoder(), ['ring-type']),
    ('oh_spore_print_color', OneHotEncoder(), ['spore-print-color']),
    ('oh_population', OneHotEncoder(), ['population']),
    ('oh_habitat', OneHotEncoder(), ['habitat']),
    ('oe_ring_number', OrdinalEncoder(categories=[['n', 'o', 't']]), ['ring-number']),
]

In [ ]:
ct = ColumnTransformer(transformers, remainder='passthrough')

In [ ]:
x = np.array(ct.fit_transform(x).todense())

## KNN

Aqui é avaliado o desempenho do classificador KNN, usando validação cruzada em dois níveis. No primeiro nível, a validação cruzada é feita em 10 vias,
enquanto no segundo nível é feita em 5 vias.

In [ ]:
def do_cv_knn(x, y, cv_splits, ks):

    #usa o protocolo de validação cruzada estratificada
    skf = StratifiedKFold(n_splits=cv_splits, shuffle=True, random_state=1)

    acuracias = []

    ## cria progress bar
    pgb = tqdm(total=cv_splits, desc='Folds avaliados')
    
    #a função split retorna os índices das instâncias que devem ser usadas para o treinamento e o teste.
    for idx_treino, idx_teste in skf.split(x, y):
        
        #extrai as instâncias de treinamento de acordo com os índices fornecidos pelo skf.split
        x_treino = x[idx_treino]
        y_treino = y[idx_treino]
        
        #extrai as instâncias de teste de acordo com os índices fornecidos pelo skf.split
        x_teste = x[idx_teste]
        y_teste = y[idx_teste]
        
        #coloca todas as variáveis na mesma escala, usando o conjunto de treinamento para calcular os parâmetros da escala
        ss = StandardScaler()
        ss.fit(x_treino)
        x_treino = ss.transform(x_treino)
        x_teste = ss.transform(x_teste)
        
        #combinações de parâmetros otimizados, otimizando o número de vizinhos mais próximos para o knn (k)
        params = {'n_neighbors' : range(1,30,2)}
        #instancia um KNN com parâmetros padrão
        knn = KNeighborsClassifier()
        #instancia um GridSearchCV com ks vias.
        knn = GridSearchCV(knn, params, cv=StratifiedKFold(n_splits=ks))    #busca exaustiva para otimização de hiperparâmetro por validação cruzada
        #realiza a otimização dos hiperparâmetros e treina o modelo final com a melhor combinação de hiperparâmetros com todos os dados de treinamento
        knn.fit(x_treino, y_treino)
        
        #calcula a acurácia no conjunto de testes desta iteração e salva na lista.
        acuracias.append(accuracy_score(y_teste, knn.predict(x_teste)))

        #atualiza o progress bar
        pgb.update(1)
        
    #fecha o progress bar
    pgb.close()
        
    return acuracias

In [ ]:
accs_knn = do_cv_knn(x, y.values, 10, 5)

Folds avaliados:   0%|          | 0/10 [00:00<?, ?it/s]

In [ ]:
#calcula as estatísticas da validação cruzada, que nos dá uma confiança que, na média, este é o desempenho esperado do classificador no mundo real
print(' Acurácia Máxima: %.3f\n Acurácia Mínima: %.3f' % ( max(accs_knn), min(accs_knn)))
print(' Acurácia Média: %.3f\n Desvio Padrão: %.3f \n' % (np.mean(accs_knn), np.std(accs_knn)))

 Acurácia Máxima: 0.990
 Acurácia Mínima: 0.940
 Acurácia Média: 0.967
 Desvio Padrão: 0.018 



## CSV

Aqui é avaliado o desempenho do classificador CSV, da mesma forma que foi avaliado anteriormente.

In [ ]:
def do_cv_svm(x, y, cv_splits, ks, Cs=[1], gammas=["scale"]):

    ## validação cruzada estratificada
    skf = StratifiedKFold(n_splits=cv_splits, shuffle=True, random_state=1)

    acuracias = []

    ## cria progress bar
    pgb = tqdm(total=cv_splits, desc='Folds avaliados')
    
    for treino_idx, teste_idx in skf.split(x, y):
        
        #extrai as instâncias de treinamento de acordo com os índices fornecidos pelo skf.split
        x_treino = x[treino_idx]
        y_treino = y[treino_idx]

        #extrai as instâncias de teste de acordo com os índices fornecidos pelo skf.split
        x_teste = x[teste_idx]
        y_teste = y[teste_idx]

        #coloca todas as variáveis na mesma escala, usando o conjunto de treinamento para calcular os parâmetros da escala
        ss = StandardScaler()
        ss.fit(x_treino)
        x_treino = ss.transform(x_treino)
        x_teste = ss.transform(x_teste)

        #combinações de parâmetros otimizados, de forma a selecionar a melhor combinação de C e γ
        params = {
            "C": Cs,
            "gamma": gammas
        }
        #instancia uma SVM usando o kernel rbf
        svm = SVC(kernel="rbf")
        #instancia um GridSearchCV com ks vias
        svm = GridSearchCV(svm, params, n_jobs=2, cv=StratifiedKFold(ks))
        #realiza a otimização dos hiperparâmetros e treina o modelo final com a melhor combinação de hiperparâmetros com todos os dados de treinamento
        svm.fit(x_treino, y_treino)

        pred = svm.predict(x_teste)

        #calcula a acurácia no conjunto de testes desta iteração e salva na lista.
        acuracias.append(accuracy_score(y_teste, pred))
        
        #atualiza o progress bar
        pgb.update(1)
        
    #fecha o progress bar
    pgb.close()
    
    return acuracias

In [ ]:
accs_svm = do_cv_svm(x, y.values, 10, 5, np.logspace(-3, 4, 8), np.logspace(-3, 4, 8))

Folds avaliados:   0%|          | 0/10 [00:00<?, ?it/s]

In [ ]:
print(' Acurácia Máxima: %.3f\n Acurácia Mínima: %.3f' % ( max(accs_svm), min(accs_svm)))
print(' Acurácia Média: %.3f\n Desvio Padrão: %.3f \n' % (np.mean(accs_svm), np.std(accs_svm)))

 Acurácia Máxima: 0.990
 Acurácia Mínima: 0.880
 Acurácia Média: 0.946
 Desvio Padrão: 0.029 



## Teste T

Para verificar se os resultados obtidos com o classificador KNN e com a SVM são estatisticamente diferentes, é feito o Teste-T (teste de hipótese nula). 

In [ ]:
ttest_ind_from_stats(np.mean(accs_knn), np.std(accs_knn), 10, np.mean(accs_svm), np.std(accs_svm), 10)

Ttest_indResult(statistic=1.9625311167671824, pvalue=0.06534760174702572)

In [ ]:
def rejeitar_hip_nula(med1, desv1, n1, med2, desv2, n2, alpha=0.05):
  _, p = ttest_ind_from_stats(med1, desv1, n1, med2, desv2, n2)
  return p <= alpha, p

In [ ]:
rejeitar_hip_nula(np.mean(accs_knn), np.std(accs_knn), 10, np.mean(accs_svm), np.std(accs_svm), 10)

(False, 0.06534760174702572)

Como resultado, tem-se que não é possível rejeitar a hipótese nula, ou seja, ambos classificadores não possuem uma diferença significativa em seus desempenhos. Ou seja, não importa se será usado o classificador KNN ou a SVM, pois ambos apresentam acurácias muito próximas.

## Pergunta

**Você usaria algum classificador que criou para decidir se comeria ou não um cogumelo classificado por ele?**

Considerando que a acurácia mínima do KNN foi de 0.94 e a da SVM foi de 0.88, eu consideraria utilizar o classificador KNN para decidir se comeria ou não um cogumelo classificado por ele, considerando que as chances são mais baixas de eu consumir um cogumelo que era venenoso, mas foi considerado como não venenoso, tendo menos risco à minha saúde.